In [1]:
import MergingBiz
import RemoveInvalidCoordinates
import FilterNoAddrInfo
import fix_NAICS_codes


In [2]:
# ExtractOpenTabOutput.main()

In [3]:
MergingBiz.main()

----------------------------------------------------------------------------------------------------------------------------
Merging files: 100%|██████████| 61/61 [00:04<00:00, 12.87it/s]
File saved to /home/jovyan/ODBiz/3-Merging/double_check/csv_rows.csv
File saved to /home/jovyan/ODBiz/3-Merging/double_check/csv_rows_sorted.csv
Reordering columns...
Done in 1 s
Standardizing Province Names...
File saved to /home/jovyan/ODBiz/3-Merging/double_check/standardized_province_names.csv
Done in 1 s
Standardizing Country Names...
Done in 0 s
Making Postal Codes consistent...
Done in 0 s
Fixing hyphens...
Done in 1 s
Creating temp column:
Capitalize values and remove punctuation characters: 100%|██████████| 9/9 [00:11<00:00,  1.24s/it]
Fill blanks with NULL...
Done in 4 s
Appending dup_key columns together and assign to temp column: 100%|██████████| 9/9 [00:02<00:00,  3.97it/s]
Applying hashing to get new indicies...
Filling in geo_source column
Marking Duplicates...
Done in 0 s
Dropping dupl

In [4]:
RemoveInvalidCoordinates.main()

Loading data: 100%|██████████| 81/81 [00:04<00:00, 16.62it/s]

Fixing invalid coordinates and marking valid ones
Done in 0 s
Forcing invalid coords to be blank
Affected rows saved to /home/jovyan/ODBiz/3-Merging/double_check/invalid_coords_affected_rows.csv
Done in 10 s
File saved to /home/jovyan/ODBiz/3-Merging/double_check/coord_ranges.csv
df saved to /home/jovyan/ODBiz/3-Merging/output/2-ODBiz_merged_2022-08-12.csv


In [5]:
FilterNoAddrInfo.main()

Loading data: 100%|██████████| 9/9 [00:04<00:00,  1.93it/s]
Filling in NA as empty string
Done in 4 s
Concatenating address info
Done in 7 s
Checking for blanks
Done in 0 s
Dropping entries with insufficient address info
Saving df_no_info to /home/jovyan/ODBiz/3-Merging/double_check/no_addr_info.csv
Done in 1 s
Saving df to /home/jovyan/ODBiz/3-Merging/output/3-ODBiz_merged_2022-08-12.csv
Done in 12 s
df saved to /home/jovyan/ODBiz/3-Merging/output/3-ODBiz_merged_2022-08-12.csv


In [6]:
fix_NAICS_codes.main()

Loading data: 100%|██████████| 9/9 [00:04<00:00,  1.94it/s]
Loaded in        803584 rows
Resulting df has 803584 rows
File saved to /home/jovyan/ODBiz/3-Merging/output/4-ODBiz_merged_2022-08-12.csv
Execution finished in 17 s


In [7]:
# Move the most recent merged file to 4-Parsing
import re 
import os 
import shutil
import pandas as pd
from tqdm import tqdm
from datetime import datetime
from dateutil.parser import parse

# Find the latest merged file
most_recent_date = datetime.min
most_recent_iter = 0
merged_output_dir = '/home/jovyan/ODBiz/3-Merging/output'
parsing_input_dir = '/home/jovyan/ODBiz/4-Parsing/input'
for f in os.listdir(merged_output_dir):
    if f.endswith('.csv'):
        date_search = re.search('(\d*)-ODBiz_merged_(\d{4}-\d{2}-\d{2})', f)
        if date_search:
            iteration = date_search.group(1)
            file_date = parse(date_search.group(2))
            if file_date > most_recent_date:
                most_recent_date = (file_date)
                most_recent_iter = iteration
            elif file_date == most_recent_date and iteration > most_recent_iter:
                most_recent_iter = iteration

most_recent_filename = f'{most_recent_iter}-ODBiz_merged_{most_recent_date.date()}.csv'
most_recent_filepath = f'{merged_output_dir}/{most_recent_filename}'

# Update the parsing/input directory to have the latest merged file
if os.path.exists(parsing_input_dir):
    shutil.rmtree(parsing_input_dir)
os.makedirs(parsing_input_dir)
output_filepath = f'{parsing_input_dir}/ODBiz_Merged.csv'
 
# # Copy over only a subset of the df
# chunksize = 100000
# for df in pd.read_csv(most_recent_filepath, chunksize=chunksize, dtype = str):
#     df.to_csv(output_filepath, index = False)
#     print(f'A subset of the most recent merged file has been saved to {output_filepath}')
#     break

# Copy over the entire df
shutil.copyfile(most_recent_filepath, output_filepath)
print(f'most_recent_filepath = {most_recent_filepath}')
print(f'Most recent merged file saved to {output_filepath}')

Most recent merged file saved to /home/jovyan/ODBiz/4-Parsing/input/ODBiz_Merged.csv
